# Topic Modelling BERT

## Import and Installation Requirements

In [ ]:
!pip install bertopic

     |████████████████████████████████| 57 kB 1.4 MB/s 
     |████████████████████████████████| 6.4 MB 7.6 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 86 kB 469 kB/s 
     |████████████████████████████████| 78 kB 7.1 MB/s 
  Using cached numpy-1.21.5-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (15.7 MB)
     |████████████████████████████████| 26.5 MB 1.5 MB/s 
     |████████████████████████████████| 3.4 MB 34.4 MB/s 
     |████████████████████████████████| 6.8 MB 37.0 MB/s 
     |████████████████████████████████| 1.2 MB 45.5 MB/s 
     |████████████████████████████████| 67 kB 4.9 MB/s 
     |████████████████████████████████| 636 kB 44.5 MB/s 
     |████████████████████████████████| 3.3 MB 37.9 MB/s 
     |████████████████████████████████| 895 kB 47.1 MB/s 
     |████████████████████████████████| 1.1 MB 58.6 MB/s 
  Created wheel for hdbscan: file

In [ ]:
!pip install bertopic[visualization]

In [ ]:
!pip install emoji --upgrade
!pip install plotly==4.*
!python -m spacy download en_core_web_lg

     |████████████████████████████████| 174 kB 4.2 MB/s 
  Created wheel for emoji: filename=emoji-1.6.3-py3-none-any.whl size=170298 sha256=d48342e1931b185b4f7e72c346a2f45c45eaf72f9dc33cc7ab7fecc239665a83
  Stored in directory: /root/.cache/pip/wheels/03/8b/d7/ad579fbef83c287215c0caab60fb0ae0f30c4d7ce5f580eade
Successfully built emoji
     |████████████████████████████████| 13.2 MB 4.2 MB/s 
  Attempting uninstall: plotly
    Found existing installation: plotly 5.5.0
    Uninstalling plotly-5.5.0:
      Successfully uninstalled plotly-5.5.0
     |████████████████████████████████| 827.9 MB 1.7 MB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.2.5-py3-none-any.whl size=829180942 sha256=ceb145dad54676355beb18fc3524f2ee72dc75027542152f971824f86de5e423
  Stored in directory: /tmp/pip-ephem-wheel-cache-b8q6au_s/wheels/11/95/ba/2c36cc368c0bd339b44a791c2c1881a1fb714b78c29a4cb8f5
Successfully built en-core-web-lg
✔ Download and installation successful
You can now load the mod

In [ ]:
# RESTART RUNTIME!!!!!!!!!!!!!!!

In [ ]:
# re-check
import pandas as pd
print(pd.__version__) # has to be 1.3.1 otherwise install it

1.1.5


In [ ]:
# dependencies
import pandas as pd
import io
from google.colab import files
from collections import Counter
import nltk
import random
import json

import warnings
warnings.simplefilter('ignore')
from itertools import chain
# import io to get access to io functions
# load library to upload data in colab
from collections import Counter
# Load the regular expression library
import re
# text
import emoji
import regex

# Required Libraries

#Base and Cleaning 
# import requests
import numpy as np
import string

#Visualizations
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt 

#Natural Language Processing (NLP)
# import pyLDAvis
# import pyLDAvis.gensim_models as gensimvis
import spacy
# import gensim
# from gensim.models.ldamulticore import LdaMulticore
# from gensim import corpora, models
# from spacy.tokenizer import Tokenizer
# from gensim.corpora import Dictionary
# from gensim.models.coherencemodel import CoherenceModel
# from gensim.parsing.preprocessing import STOPWORDS as SW
# from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
# from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
# from sklearn.model_selection import GridSearchCV
from pprint import pprint
from wordcloud import STOPWORDS
stopwords = set(STOPWORDS)

In [ ]:
# Load spacy
# Make sure to restart the runtime after running installations and libraries tab
nlp = spacy.load('en_core_web_lg')

In [ ]:
# ignore warnings
import warnings
warnings.filterwarnings('ignore')

## Load Data

In [ ]:
# call function to select file to upload --> PolPred_Data.csv
uploaded = files.upload()

Saving data_compact_all_types.csv to data_compact_all_types.csv


In [ ]:
# This file can be exchanged based on new data to be used (this is just the reference to the data set that was used in the last run)
used_data_file = 'data_compact_all_types.csv'

In [ ]:
# read uploaded file
d = pd.read_csv(io.BytesIO(uploaded[used_data_file]), dtype=str)
# Dataset is now stored in a Pandas Dataframe

In [ ]:
# check data
d

,Unnamed: 0,Unnamed: 0.1,topic_theme,author_id,created_at,geo,tweet_id,lang,like_count,quote_count,reply_count,retweet_count,source,text,retweeted
0,0,0,climate change,1236762045082673153,2021-12-13 18:23:43+00:00,NaN,1470459432874164225,en,0,0,0,23,Twitter for iPhone,"RT @FeministWeiner: ""Climate Change"" Justin Tr...",retweeted
1,1,1,climate change,1333880654619021313,2021-12-13 18:23:43+00:00,NaN,1470459431926173696,en,0,0,0,3407,Twitter for Android,RT @DanielTurnerPTF: The top 5 deadliest torna...,retweeted
2,2,2,climate change,768163767376936960,2021-12-13 18:23:42+00:00,NaN,1470459429275553801,en,0,0,0,3624,Twitter for iPhone,RT @disclosetv: JUST IN - Russia vetoes UN res...,retweeted
3,3,3,climate change,1537559256,2021-12-13 18:23:41+00:00,NaN,1470459426666692614,en,0,0,0,0,Twitter for Android,Russia's not falling for that crap! IN - Russi...,retweeted
4,4,4,climate change,1214978408922222592,2021-12-13 18:23:41+00:00,NaN,1470459425513095169,en,0,0,0,0,Twitter Web App,It’s exciting to see more incentives to suppor...,quoted
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167267,4401,4401,ukraine conflict,2787089534,2021-12-26 19:39:54+00:00,NaN,1475189647252807684,en,0,0,0,25,Twitter for Android,RT @PrplScrubsCHPN: @RonFilipkowski @Judson4Co...,retweeted
167268,4402,4402,ukraine conflict,3053193701,2021-12-26 19:38:24+00:00,NaN,1475189268989554690,en,0,0,0,28,Twitter for iPad,RT @SkyNews: Speaking at his annual news confe...,retweeted
167269,4403,4403,ukraine conflict,122756897,2021-12-26 19:35:44+00:00,NaN,1475188601403150336,en,0,0,1,0,Twitter for Android,@Morearighty It's easy. I'm a Russian immigran...,replied_to
167270,4404,4404,ukraine conflict,1400022519613214720,2021-12-26 19:34:58+00:00,NaN,1475188408427421701,en,0,0,0,19,Twitter for Android,RT @France24_en: @RobParsonsF24 The Wagner Gro...,retweeted


In [ ]:
# duplicates need to be removed
print(len(d))
d = d.drop_duplicates(subset='text')
len(d)

167272


76000

In [ ]:
# check data
d.describe()

,Unnamed: 0,Unnamed: 0.1,topic_theme,author_id,created_at,geo,tweet_id,lang,like_count,quote_count,reply_count,retweet_count,source,text,retweeted
count,76000,76000,76000,75997,75999,580,76000,75997,75999,75999,75999,75999,75999,75999,76000
unique,19859,19859,10,54562,54080,423,76000,1,321,36,76,1298,1158,75999,3
top,1,1,stock market,1162211777557356545,2021-12-13 16:15:00+00:00,{'place_id': '3b77caf94bfc81fe'},1478309937965060098,en,0,0,0,0,Twitter Web App,RT @FootballlForAll: 🚨⚽️ | BREAKING: Mancheste...,retweeted
freq,10,10,11378,398,23,11,1,75997,56390,73859,63990,47544,21485,1,42529


In [ ]:
# check data
d.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'topic_theme', 'author_id', 'created_at',
       'geo', 'tweet_id', 'lang', 'like_count', 'quote_count', 'reply_count',
       'retweet_count', 'source', 'text', 'retweeted'],
      dtype='object')

## Data Cleaning

In [ ]:
# Remove unnecessary columsn
df = d.drop(columns=['Unnamed: 0', 'Unnamed: 0.1', 'topic_theme', 'author_id', 'created_at',
       'geo', 'tweet_id', 'lang', 'like_count', 'quote_count', 'reply_count',
       'retweet_count', 'source', 'retweeted'], axis=1)
df

,text
0,"RT @FeministWeiner: ""Climate Change"" Justin Tr..."
1,RT @DanielTurnerPTF: The top 5 deadliest torna...
2,RT @disclosetv: JUST IN - Russia vetoes UN res...
3,Russia's not falling for that crap! IN - Russi...
4,It’s exciting to see more incentives to suppor...
...,...
167266,@andybeega @SpaghettiKozak @soonergrunt @tmbhm...
167268,RT @SkyNews: Speaking at his annual news confe...
167269,@Morearighty It's easy. I'm a Russian immigran...
167270,RT @France24_en: @RobParsonsF24 The Wagner Gro...


In [ ]:
# check data
print(len(df))
df = df[df['text'].notna()]
len(df)

76000


75999

In [ ]:
# Utility methods to identify special characters and terms
def give_emoji_free_text(text):
    """
    Removes emoji's from tweets
    Accepts:
        Text (tweets)
    Returns:
        Text (emoji free tweets)
    """
    emoji_list = [c for c in text if c in emoji.UNICODE_EMOJI['en']]
    clean_text = ' '.join([str for str in text.split() if not any(i in str for i in emoji_list)])
    return clean_text

def url_free_text(text):
    '''
    Cleans text from urls
    '''
    text = re.sub(r'http\S+', '', text)
    return text

def at_free_text(text):
    # \S*@\S*\s?
    text = re.sub(r'\S*@\S*\s?', '', text)
    return text

In [ ]:
# Apply the function above and get tweets free of emoji's
call_emoji_free = lambda x: give_emoji_free_text(x)

# Apply `call_emoji_free` which calls the function to remove all emoji's
df['emoji_free_text'] = df['text'].apply(call_emoji_free)

# Create a new column with url free tweets
df['at_free_text'] = df['emoji_free_text'].apply(at_free_text)

# Create a new column with url free tweets
df['url_free_text'] = df['at_free_text'].apply(url_free_text)


In [ ]:
# check data
df

,text,emoji_free_text,at_free_text,url_free_text
0,"RT @FeministWeiner: ""Climate Change"" Justin Tr...","RT @FeministWeiner: ""Climate Change"" Justin Tr...","RT ""Climate Change"" Justin Trudeau is so Liber...","RT ""Climate Change"" Justin Trudeau is so Liber..."
1,RT @DanielTurnerPTF: The top 5 deadliest torna...,RT @DanielTurnerPTF: The top 5 deadliest torna...,RT The top 5 deadliest tornadoes in America al...,RT The top 5 deadliest tornadoes in America al...
2,RT @disclosetv: JUST IN - Russia vetoes UN res...,RT @disclosetv: JUST IN - Russia vetoes UN res...,RT JUST IN - Russia vetoes UN resolution to re...,RT JUST IN - Russia vetoes UN resolution to re...
3,Russia's not falling for that crap! IN - Russi...,Russia's not falling for that crap! IN - Russi...,Russia's not falling for that crap! IN - Russi...,Russia's not falling for that crap! IN - Russi...
4,It’s exciting to see more incentives to suppor...,It’s exciting to see more incentives to suppor...,It’s exciting to see more incentives to suppor...,It’s exciting to see more incentives to suppor...
...,...,...,...,...
167266,@andybeega @SpaghettiKozak @soonergrunt @tmbhm...,@andybeega @SpaghettiKozak @soonergrunt @tmbhm...,"As I said earlier, gun to my head, I guess I'd...","As I said earlier, gun to my head, I guess I'd..."
167268,RT @SkyNews: Speaking at his annual news confe...,RT @SkyNews: Speaking at his annual news confe...,"RT Speaking at his annual news conference, Rus...","RT Speaking at his annual news conference, Rus..."
167269,@Morearighty It's easy. I'm a Russian immigran...,@Morearighty It's easy. I'm a Russian immigran...,It's easy. I'm a Russian immigrant who doesn't...,It's easy. I'm a Russian immigrant who doesn't...
167270,RT @France24_en: @RobParsonsF24 The Wagner Gro...,RT @France24_en: @RobParsonsF24 The Wagner Gro...,RT The Wagner Group has caused controversy thr...,RT The Wagner Group has caused controversy thr...


In [ ]:
df['prepared_text'] = df.apply(lambda row: " ".join(re.sub("[^a-zA-Z]+", " ", row.url_free_text).split()), 1)
df

,text,emoji_free_text,at_free_text,url_free_text,prepared_text
0,"RT @FeministWeiner: ""Climate Change"" Justin Tr...","RT @FeministWeiner: ""Climate Change"" Justin Tr...","RT ""Climate Change"" Justin Trudeau is so Liber...","RT ""Climate Change"" Justin Trudeau is so Liber...",RT Climate Change Justin Trudeau is so Liberal...
1,RT @DanielTurnerPTF: The top 5 deadliest torna...,RT @DanielTurnerPTF: The top 5 deadliest torna...,RT The top 5 deadliest tornadoes in America al...,RT The top 5 deadliest tornadoes in America al...,RT The top deadliest tornadoes in America all ...
2,RT @disclosetv: JUST IN - Russia vetoes UN res...,RT @disclosetv: JUST IN - Russia vetoes UN res...,RT JUST IN - Russia vetoes UN resolution to re...,RT JUST IN - Russia vetoes UN resolution to re...,RT JUST IN Russia vetoes UN resolution to reco...
3,Russia's not falling for that crap! IN - Russi...,Russia's not falling for that crap! IN - Russi...,Russia's not falling for that crap! IN - Russi...,Russia's not falling for that crap! IN - Russi...,Russia s not falling for that crap IN Russia v...
4,It’s exciting to see more incentives to suppor...,It’s exciting to see more incentives to suppor...,It’s exciting to see more incentives to suppor...,It’s exciting to see more incentives to suppor...,It s exciting to see more incentives to suppor...
...,...,...,...,...,...
167266,@andybeega @SpaghettiKozak @soonergrunt @tmbhm...,@andybeega @SpaghettiKozak @soonergrunt @tmbhm...,"As I said earlier, gun to my head, I guess I'd...","As I said earlier, gun to my head, I guess I'd...",As I said earlier gun to my head I guess I d r...
167268,RT @SkyNews: Speaking at his annual news confe...,RT @SkyNews: Speaking at his annual news confe...,"RT Speaking at his annual news conference, Rus...","RT Speaking at his annual news conference, Rus...",RT Speaking at his annual news conference Russ...
167269,@Morearighty It's easy. I'm a Russian immigran...,@Morearighty It's easy. I'm a Russian immigran...,It's easy. I'm a Russian immigrant who doesn't...,It's easy. I'm a Russian immigrant who doesn't...,It s easy I m a Russian immigrant who doesn t ...
167270,RT @France24_en: @RobParsonsF24 The Wagner Gro...,RT @France24_en: @RobParsonsF24 The Wagner Gro...,RT The Wagner Group has caused controversy thr...,RT The Wagner Group has caused controversy thr...,RT The Wagner Group has caused controversy thr...


In [ ]:
# check data
df['prepared_text']

0         RT Climate Change Justin Trudeau is so Liberal...
1         RT The top deadliest tornadoes in America all ...
2         RT JUST IN Russia vetoes UN resolution to reco...
3         Russia s not falling for that crap IN Russia v...
4         It s exciting to see more incentives to suppor...
                                ...                        
167266    As I said earlier gun to my head I guess I d r...
167268    RT Speaking at his annual news conference Russ...
167269    It s easy I m a Russian immigrant who doesn t ...
167270    RT The Wagner Group has caused controversy thr...
167271    Lots of progressives rightfully questioning th...
Name: prepared_text, Length: 75999, dtype: object

In [ ]:
# replace short words
def remove_short_words(text):
  text = re.sub(r"\b\w{1,2}\b", "", text)
  text = " ".join(text.split()) # remove unnecessary/duplicated whitespaces
  return text

In [ ]:
df['prepared_text'] = df['prepared_text'].apply(remove_short_words)
df['prepared_text']

0         Climate Change Justin Trudeau Liberal his pron...
1         The top deadliest tornadoes America all occurr...
2         JUST Russia vetoes resolution recognize climat...
3         Russia not falling for that crap Russia vetoes...
4         exciting see more incentives support clean tra...
                                ...                        
167266    said earlier gun head guess rather Ukraine win...
167268    Speaking his annual news conference Russian Pr...
167269    easy Russian immigrant who doesn support Russi...
167270    The Wagner Group has caused controversy throug...
167271    Lots progressives rightfully questioning the h...
Name: prepared_text, Length: 75999, dtype: object

In [ ]:
# duplicates need to be removed
print(len(df))
df = df.drop_duplicates(subset=['prepared_text'])
len(df)

75999


63070

In [ ]:
# store data in a format the model algorithm is expecting it (list of texts)
titles = df['prepared_text'].to_list()
titles

['Climate Change Justin Trudeau Liberal his pronouns are maybe sometimes',
 'The top deadliest tornadoes America all occurred between and Today climate change crazies weren',
 'JUST Russia vetoes resolution recognize climate change threat global security',
 'Russia not falling for that crap Russia vetoes resolution recognize climate change threat global security',
 'exciting see more incentives support clean transportation doing our part expanding the charging network for electric vehicles and adopting new standards bring more zero emission choices Washington car buyers',
 'Absolute There are less tornadoes now than years ago Nothing with your climate change mandat',
 'have idea what caused those tornadoes thinking more like Jehova God Armies But climate change does need addressed need invest the American Helion Corporation amp These daunting problems require solutions with public spearhead investment',
 'Manchin cites blind trust justify climate votes But much income from his family c

In [ ]:
# check data
len(titles)

63070

In [ ]:
# load the BERTopic library
from bertopic import BERTopic

In [ ]:
# create BERTopic model
# Note: The parameters can be adjusted as required for hyperparameter tuning
# As the calculation of these models took a lot of time, I have not created a utility method to automate the hyperparameter tuning here
# I basically did the hyperparameter tuning here manually by exchanging the parameters and run the code again --> hence, this cannot be shown here
topic_model = BERTopic(min_topic_size=350, n_gram_range=(1,3), verbose=True, calculate_probabilities=True)
topics, probs = topic_model.fit_transform(titles)

Batches:   0%|          | 0/1971 [00:00<?, ?it/s]

2022-01-26 10:28:04,134 - BERTopic - Transformed documents to Embeddings
2022-01-26 10:29:34,906 - BERTopic - Reduced dimensionality with UMAP
2022-01-26 10:30:04,777 - BERTopic - Clustered UMAP embeddings with HDBSCAN


In [ ]:
# check data
len(topics)

63070

In [ ]:
# check data
len(probs)

63070

In [ ]:
# check data
print(len(topics))
print(len(probs))

63070
63070


In [ ]:
# check data
freq = topic_model.get_topic_info()
freq.head(100)

,Topic,Count,Name
0,0,9235,0_racism_the_and_you
1,1,8956,1_covid_the_and_for
2,2,8902,2_movie_the_and_the movie
3,3,7133,3_data_privacy_and_the
4,4,6907,4_stock_market_stock market_the
5,5,5673,5_gaming_the_and_game
6,6,5506,6_climate_change_climate change_the
7,7,5061,7_electric_vehicle_electric vehicle_the
8,8,3223,8_olympic_games_the_olympic games
9,9,1214,9_ukraine_conflict_russia_the


In [ ]:
# check data
len(freq)

12

In [ ]:
# check data
topic_nr = freq.iloc[4]["Topic"] # select a frequent topic
topic_model.get_topic(topic_nr)

[('stock', 0.058772796235241634),
 ('market', 0.0585664034811978),
 ('stock market', 0.047613752043748354),
 ('the', 0.03687665790315058),
 ('the stock', 0.028041711075597053),
 ('the stock market', 0.025930235674327402),
 ('and', 0.018239220707054648),
 ('for', 0.012350454835139125),
 ('you', 0.012193842597916827),
 ('stocks', 0.01047376857334356)]

In [ ]:
topic_model.visualize_topics()

In [ ]:
df['tokens'] = df['prepared_text'].apply(str.split)
df

,text,emoji_free_text,at_free_text,url_free_text,prepared_text,tokens
0,"RT @FeministWeiner: ""Climate Change"" Justin Tr...","RT @FeministWeiner: ""Climate Change"" Justin Tr...","RT ""Climate Change"" Justin Trudeau is so Liber...","RT ""Climate Change"" Justin Trudeau is so Liber...",Climate Change Justin Trudeau Liberal his pron...,"[Climate, Change, Justin, Trudeau, Liberal, hi..."
1,RT @DanielTurnerPTF: The top 5 deadliest torna...,RT @DanielTurnerPTF: The top 5 deadliest torna...,RT The top 5 deadliest tornadoes in America al...,RT The top 5 deadliest tornadoes in America al...,The top deadliest tornadoes America all occurr...,"[The, top, deadliest, tornadoes, America, all,..."
2,RT @disclosetv: JUST IN - Russia vetoes UN res...,RT @disclosetv: JUST IN - Russia vetoes UN res...,RT JUST IN - Russia vetoes UN resolution to re...,RT JUST IN - Russia vetoes UN resolution to re...,JUST Russia vetoes resolution recognize climat...,"[JUST, Russia, vetoes, resolution, recognize, ..."
3,Russia's not falling for that crap! IN - Russi...,Russia's not falling for that crap! IN - Russi...,Russia's not falling for that crap! IN - Russi...,Russia's not falling for that crap! IN - Russi...,Russia not falling for that crap Russia vetoes...,"[Russia, not, falling, for, that, crap, Russia..."
4,It’s exciting to see more incentives to suppor...,It’s exciting to see more incentives to suppor...,It’s exciting to see more incentives to suppor...,It’s exciting to see more incentives to suppor...,exciting see more incentives support clean tra...,"[exciting, see, more, incentives, support, cle..."
...,...,...,...,...,...,...
167266,@andybeega @SpaghettiKozak @soonergrunt @tmbhm...,@andybeega @SpaghettiKozak @soonergrunt @tmbhm...,"As I said earlier, gun to my head, I guess I'd...","As I said earlier, gun to my head, I guess I'd...",said earlier gun head guess rather Ukraine win...,"[said, earlier, gun, head, guess, rather, Ukra..."
167268,RT @SkyNews: Speaking at his annual news confe...,RT @SkyNews: Speaking at his annual news confe...,"RT Speaking at his annual news conference, Rus...","RT Speaking at his annual news conference, Rus...",Speaking his annual news conference Russian Pr...,"[Speaking, his, annual, news, conference, Russ..."
167269,@Morearighty It's easy. I'm a Russian immigran...,@Morearighty It's easy. I'm a Russian immigran...,It's easy. I'm a Russian immigrant who doesn't...,It's easy. I'm a Russian immigrant who doesn't...,easy Russian immigrant who doesn support Russi...,"[easy, Russian, immigrant, who, doesn, support..."
167270,RT @France24_en: @RobParsonsF24 The Wagner Gro...,RT @France24_en: @RobParsonsF24 The Wagner Gro...,RT The Wagner Group has caused controversy thr...,RT The Wagner Group has caused controversy thr...,The Wagner Group has caused controversy throug...,"[The, Wagner, Group, has, caused, controversy,..."


In [ ]:
topic_dic = topic_model.get_topics()
topic_dic

{-1: [('stock', 0.032063140704362864),
  ('market ebook', 0.03133144895132655),
  ('visit the stock', 0.03133144895132655),
  ('ebook website', 0.03133144895132655),
  ('stock market ebook', 0.03133144895132655),
  ('market ebook website', 0.03133144895132655),
  ('more info about', 0.031270886057040564),
  ('info about', 0.031181704341545678),
  ('ebook', 0.030897846161528675),
  ('visit the', 0.03081642346229779)],
 0: [('racism', 0.055034969968020465),
  ('the', 0.02629064077493449),
  ('and', 0.022671933714540342),
  ('you', 0.018737590630658767),
  ('that', 0.01821083147862422),
  ('white', 0.014172796468642578),
  ('people', 0.013414262800797714),
  ('not', 0.012237140839965768),
  ('this', 0.011648483817642646),
  ('are', 0.011628432213140109)],
 1: [('covid', 0.05702053737489012),
  ('the', 0.0284935775964477),
  ('and', 0.018887840040830602),
  ('for', 0.013952812569155128),
  ('you', 0.012649955427558186),
  ('are', 0.012372276953283065),
  ('that', 0.011989676770934182),
  (

In [ ]:
# coherence
# In order to calculate the coherence, gensim libraries have to be used
import gensim.corpora as corpora
from gensim.models.coherencemodel import CoherenceModel

bert_dic = corpora.Dictionary(df['tokens'])
bert_corpus = [bert_dic.doc2bow(text) for text in df['tokens']]

In [ ]:
# Additionally the topics created by the BERTopic model need sometimes some adjustments in order to get used correclty by the gensim algorithm
# Hence, I perform here this adjustment
list_of_topics = []
for key, value in topic_dic.items():
  topic_list = []
  for entry in value:
    # print(entry[0].split())
    if len(entry[0].split()) == 1:
      # ignore n-grams here
      if len(bert_dic.doc2bow(entry[0].split())) > 0:
        topic_list.append(entry[0])
  list_of_topics.append(topic_list)
list_of_topics

[['stock', 'ebook'],
 ['racism',
  'the',
  'and',
  'you',
  'that',
  'white',
  'people',
  'not',
  'this',
  'are'],
 ['covid', 'the', 'and', 'for', 'you', 'are', 'that', 'have', 'with', 'not'],
 ['movie', 'the', 'and', 'this', 'that', 'was', 'you', 'for', 'like'],
 ['data', 'privacy', 'and', 'the', 'for', 'your', 'security', 'you', 'that'],
 ['stock', 'market', 'the', 'and', 'for', 'you', 'stocks'],
 ['gaming',
  'the',
  'and',
  'game',
  'for',
  'you',
  'games',
  'nft',
  'with',
  'this'],
 ['climate', 'change', 'the', 'and', 'that', 'are', 'for', 'you', 'this'],
 ['electric',
  'vehicle',
  'the',
  'and',
  'for',
  'charging',
  'vehicles',
  'car',
  'tesla'],
 ['olympic', 'games', 'the', 'beijing', 'winter', 'and', 'for'],
 ['ukraine',
  'conflict',
  'russia',
  'the',
  'and',
  'nato',
  'putin',
  'war',
  'russian'],
 ['price']]

In [ ]:
# check data
cm = CoherenceModel(topics=list_of_topics,texts = df['tokens'],corpus=bert_corpus, dictionary=bert_dic, coherence='c_v')
coherence = cm.get_coherence()

In [ ]:
# check data
coherence

0.530012508182687

### Store Model & Meta Data

In [ ]:
topic_model.save('bert_topic_model')

In [ ]:
import json
corpus_json = json.dumps(bert_corpus)

In [ ]:
# send file to fileshare
with open('json_corpus.txt', 'w') as outfile:
    json.dump(corpus_json, outfile)

In [ ]:
topic_model.get_topics()

{-1: [('the', 0.019897443414189378),
  ('and', 0.01405091323127068),
  ('nengi', 0.012302699754871724),
  ('for', 0.011879787840896846),
  ('you', 0.008766783287925392),
  ('amp', 0.007581458302440592),
  ('queen', 0.007214736583433862),
  ('with', 0.00709725114807458),
  ('nengi day', 0.007042716923391284),
  ('stock', 0.007038846554476803)],
 0: [('covid', 0.019549168544432984),
  ('the', 0.018951848495836133),
  ('healthcare', 0.016567710515122028),
  ('research', 0.01618448800324245),
  ('and', 0.015168773559503098),
  ('for', 0.01237539987213645),
  ('you', 0.011310294441325485),
  ('that', 0.009212973987771569),
  ('are', 0.008914302728750078),
  ('with', 0.008280444332858917)],
 1: [('gun', 0.05186468645010787),
  ('control', 0.04545358351800708),
  ('gun control', 0.04203275397665079),
  ('the', 0.020232628921642175),
  ('and', 0.013353078644551156),
  ('you', 0.01217547711625567),
  ('that', 0.011857465355272042),
  ('abortion', 0.011124261005924509),
  ('for', 0.0105192611391

In [ ]:
import json
use_model_topics = topic_model.get_topics()
topic_json = json.dumps(use_model_topics)
print(topic_json)

{"-1": [["the", 0.019897443414189378], ["and", 0.01405091323127068], ["nengi", 0.012302699754871724], ["for", 0.011879787840896846], ["you", 0.008766783287925392], ["amp", 0.007581458302440592], ["queen", 0.007214736583433862], ["with", 0.00709725114807458], ["nengi day", 0.007042716923391284], ["stock", 0.007038846554476803]], "0": [["covid", 0.019549168544432984], ["the", 0.018951848495836133], ["healthcare", 0.016567710515122028], ["research", 0.01618448800324245], ["and", 0.015168773559503098], ["for", 0.01237539987213645], ["you", 0.011310294441325485], ["that", 0.009212973987771569], ["are", 0.008914302728750078], ["with", 0.008280444332858917]], "1": [["gun", 0.05186468645010787], ["control", 0.04545358351800708], ["gun control", 0.04203275397665079], ["the", 0.020232628921642175], ["and", 0.013353078644551156], ["you", 0.01217547711625567], ["that", 0.011857465355272042], ["abortion", 0.011124261005924509], ["for", 0.010519261139135804], ["guns", 0.009727404056037715]], "2": [[

In [ ]:
# send file to fileshare
with open('json_topics.txt', 'w') as outfile:
    json.dump(topic_json, outfile)

In [ ]:
# create string to document the protocol of this execution
meta_data_string = 'Metadata for performed BERT Model\n------------------------------------------\n\n'

In [ ]:
from datetime import datetime
# datetime object containing current date and time
now = datetime.now()

In [ ]:
meta_data_string = meta_data_string + f'used data file: {used_data_file}\n------------------------------------------\nexecuted on: {now} \n------------------------------------------\n'
print(meta_data_string)

Metadata for performed BERT Model
------------------------------------------

used data file: data_complete_all_types.csv
------------------------------------------
executed on: 2022-01-24 09:41:58.095974 
------------------------------------------



In [ ]:
selected_model_details = f'BERTopic - 3-gram, 350 min size, coherence {coherence}'

In [ ]:
meta_data_string = meta_data_string + f'selected model has the following parameters:\n{selected_model_details}\n------------------------------------------'
print(meta_data_string)

Metadata for performed BERT Model
------------------------------------------

used data file: data_complete_all_types.csv
------------------------------------------
executed on: 2022-01-24 09:41:58.095974 
------------------------------------------
selected model has the following parameters:
BERTopic - 3-gram, 350 min size, coherence 0.5483433174564941
------------------------------------------


In [ ]:
free_text = '\nif required\n------------------------------------------'

In [ ]:
meta_data_string = meta_data_string + free_text
print(meta_data_string)

Metadata for performed BERT Model
------------------------------------------

used data file: data_complete_all_types.csv
------------------------------------------
executed on: 2022-01-24 09:41:58.095974 
------------------------------------------
selected model has the following parameters:
BERTopic - 3-gram, 350 min size, coherence 0.5483433174564941
------------------------------------------
if required
------------------------------------------


In [ ]:
topic_dict = topic_model.get_topics()

print_topics = {}

for line in topic_dict:
  topic_list = topic_dict[line]
  words = [entry[0] for entry in topic_list]
  print_topics[line] = words

print_topics

{-1: ['the',
  'price',
  'and',
  'for',
  'nengi',
  'targets',
  'will',
  'you',
  'price targets',
  'follow price'],
 0: ['covid',
  'the',
  'healthcare',
  'research',
  'and',
  'for',
  'you',
  'that',
  'are',
  'with'],
 1: ['gun',
  'control',
  'gun control',
  'the',
  'and',
  'you',
  'that',
  'abortion',
  'for',
  'guns'],
 2: ['privacy',
  'data',
  'and',
  'the',
  'data privacy',
  'rights',
  'your',
  'for',
  'security',
  'you'],
 3: ['movie',
  'the',
  'the movie',
  'and',
  'this',
  'was',
  'that',
  'like',
  'this movie',
  'you'],
 4: ['racism',
  'the',
  'and',
  'you',
  'that',
  'white',
  'people',
  'racist',
  'black',
  'not'],
 5: ['royals',
  'the',
  'the royals',
  'and',
  'young royals',
  'young',
  'they',
  'for',
  'are',
  'that'],
 6: ['music',
  'the',
  'song',
  'and',
  'you',
  'video',
  'for',
  'slmt',
  'music video',
  'the music'],
 7: ['stock',
  'market',
  'stock market',
  'the stock',
  'the',
  'the stock marke

In [ ]:
# Getting the topics
meta_data_string = meta_data_string + '\n------------------------------------------\n------------------------------------------\n\n'
for id in print_topics: 
  topic_id = (id + 1) if id != -1 else id
  # topic_id = id + 1
  meta_data_string = meta_data_string + f'------ Topic {topic_id} ------\n'
  meta_data_string = meta_data_string + str(' : '.join(print_topics[id])) + '\n\n'
meta_data_string = meta_data_string + '\n------------------------------------------\n'
print(meta_data_string)

Metadata for performed BERT Model
------------------------------------------

used data file: data_complete_all_types.csv
------------------------------------------
executed on: 2022-01-24 09:41:58.095974 
------------------------------------------
selected model has the following parameters:
BERTopic - 3-gram, 350 min size, coherence 0.5483433174564941
------------------------------------------
if required
------------------------------------------
------------------------------------------
------------------------------------------

------ Topic -1 ------
the : and : nengi : for : you : amp : queen : with : nengi day : stock

------ Topic 1 ------
covid : the : healthcare : research : and : for : you : that : are : with

------ Topic 2 ------
gun : control : gun control : the : and : you : that : abortion : for : guns

------ Topic 3 ------
racism : the : and : you : white : that : people : racist : black : not

------ Topic 4 ------
movie : the : the movie : and : this : was : that 

In [ ]:
# send file to fileshare
file = open("meta_data.txt", "w") 
file.write(meta_data_string) 
file.close() 

## Build Dataframe with LDA Topic Probabilities

In order to compare later the human results with the model results we need to store the model assignments to the corresponding data set. Hence, this assignment is done in the following cells and closed by the storage of thsi file.

In [ ]:
c_num_topics = []
for id in print_topics:
  l_col = (id + 1) if id != -1 else id
  c_num_topics.append(l_col)

In [ ]:
c_num_topics.append('top topic')

In [ ]:
c_num_topics

[-1,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 'top topic']

In [ ]:
df_num_topics = pd.DataFrame(columns=c_num_topics)
df_num_topics

,-1,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,top topic


In [ ]:
row_default = {str(i) : 0 for i in c_num_topics}
row_default

{'-1': 0,
 '1': 0,
 '10': 0,
 '11': 0,
 '12': 0,
 '13': 0,
 '14': 0,
 '15': 0,
 '16': 0,
 '17': 0,
 '18': 0,
 '19': 0,
 '2': 0,
 '20': 0,
 '21': 0,
 '3': 0,
 '4': 0,
 '5': 0,
 '6': 0,
 '7': 0,
 '8': 0,
 '9': 0,
 'top topic': 0}

In [ ]:
row_default['-1']

0

In [ ]:
len(probs[0])
# probs has 11 entries, but we have 12 topics (one for -1 --> undefined --> this is not given --> so I do not need a row for this --> I can ignore them --> but all to 0 in this case)

21

In [ ]:
len(titles) == len(probs) == len(topics)

True

In [ ]:
count_pref_topic_id = 0
count_max_topic_id = 0
test_value = 9
for id in range(len(titles)):
  if topics[id] == test_value:
    # print(id)
    # print(topics[id])
    if topics[id] == test_value:
      count_pref_topic_id += 1
    else:
      print(f'error {id} and {probs[id]}')
      m_v = max(probs[id])
      print(m_v)
      print('\n')
    
    if np.argmax(probs[id]) == test_value:
      count_max_topic_id += 1
    else:
      # print(f'error {id} and {probs[id]}')
      m_v = max(probs[id])
      # print(m_v)
      if m_v > 0.5:
        print(f'error {id} and {probs[id]}')
        print('CRITICAL: {m_v}')
        print('\n')


print(count_pref_topic_id)
print(count_max_topic_id)
print(count_pref_topic_id == count_max_topic_id)

6040
6029
False


In [ ]:
# next to - 1, several had a small deviation but they are all below 5% so can be ignored

In [ ]:
probs[0]

array([0.00945074, 0.00705237, 0.01669997, 0.00686846, 0.00726792,
       0.00841579, 0.00582192, 0.7420128 , 0.007163  , 0.00632169,
       0.00758445, 0.00655374, 0.00449498, 0.00835639, 0.00573097,
       0.00673003, 0.01071976, 0.00706808, 0.00574694, 0.0054246 ,
       0.00839632, 0.00676335, 0.00475986])

In [ ]:
import time
df_num_topics = pd.DataFrame()
print(len(df_num_topics))
for id in range(len(titles)):
  l_row = {}
  if id % 10000 == 0:
    print(id)
    print(l_row)
  if topics[id] == -1:
    l_row['top topic'] = topics[id]
    l_row['-1'] = 1
  else:
    l_row['top topic'] = topics[id] + 1
    l_row['-1'] = 0
  
  for idx, p in enumerate(probs[id]):
    key = idx + 1
    l_row[str(key)] = round(p, 5)
  df_num_topics = df_num_topics.append(l_row, ignore_index=True)

  if id % 10000 == 0:
    print(l_row)

len(df_num_topics)

0
0
{}
{'top topic': 8, '-1': 0, '1': 0.00945, '2': 0.00705, '3': 0.0167, '4': 0.00687, '5': 0.00727, '6': 0.00842, '7': 0.00582, '8': 0.74201, '9': 0.00716, '10': 0.00632, '11': 0.00758, '12': 0.00655, '13': 0.00449, '14': 0.00836, '15': 0.00573, '16': 0.00673, '17': 0.01072, '18': 0.00707, '19': 0.00575, '20': 0.00542, '21': 0.0084, '22': 0.00676, '23': 0.00476}
10000
{}
{'top topic': 10, '-1': 0, '1': 0.02126, '2': 0.01223, '3': 0.01001, '4': 0.00765, '5': 0.0142, '6': 0.00935, '7': 0.01893, '8': 0.01002, '9': 0.00779, '10': 0.70826, '11': 0.01286, '12': 0.01388, '13': 0.02037, '14': 0.01179, '15': 0.00948, '16': 0.0174, '17': 0.0131, '18': 0.02238, '19': 0.01327, '20': 0.01347, '21': 0.01049, '22': 0.00855, '23': 0.01325}
20000
{}
{'top topic': 1, '-1': 0, '1': 0.54525, '2': 0.02455, '3': 0.02009, '4': 0.01268, '5': 0.0273, '6': 0.01515, '7': 0.02028, '8': 0.01827, '9': 0.01506, '10': 0.03023, '11': 0.02366, '12': 0.01914, '13': 0.01843, '14': 0.02525, '15': 0.0172, '16': 0.02343, 

136861

In [ ]:
# check data
len(df)

136861

In [ ]:
# check data
df_num_topics

,-1,1,10,11,12,13,14,15,16,17,18,19,2,20,21,22,23,3,4,5,6,7,8,9,top topic
0,0.0,0.00945,0.00632,0.00758,0.00655,0.00449,0.00836,0.00573,0.00673,0.01072,0.00707,0.00575,0.00705,0.00542,0.00840,0.00676,0.00476,0.01670,0.00687,0.00727,0.00842,0.00582,0.74201,0.00716,8.0
1,0.0,0.01887,0.01220,0.01462,0.01339,0.00875,0.01572,0.01166,0.01312,0.02039,0.01371,0.01111,0.01292,0.01021,0.01765,0.01388,0.00942,0.03098,0.01422,0.01408,0.01782,0.01150,0.32470,0.01466,8.0
2,0.0,0.01862,0.01219,0.01512,0.01311,0.00879,0.01669,0.01149,0.01330,0.02156,0.01378,0.01138,0.01373,0.01064,0.01721,0.01401,0.00941,0.03526,0.01418,0.01459,0.01718,0.01147,0.34985,0.01507,8.0
3,0.0,0.01214,0.00828,0.00954,0.00897,0.00605,0.01048,0.00808,0.00861,0.01334,0.00899,0.00743,0.00900,0.00706,0.01155,0.00950,0.00645,0.01945,0.00970,0.00955,0.01195,0.00769,0.78610,0.01009,8.0
4,0.0,0.01349,0.00905,0.01051,0.01005,0.00660,0.01131,0.00893,0.00953,0.01440,0.00995,0.00816,0.00955,0.00758,0.01313,0.01049,0.00711,0.02107,0.01091,0.01031,0.01364,0.00852,0.76463,0.01109,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136856,0.0,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,19.0
136857,0.0,0.00436,0.00369,0.00738,0.00325,0.00300,0.00416,0.00212,0.01276,0.00356,0.00776,0.59365,0.00333,0.00412,0.00283,0.00240,0.00312,0.00311,0.00232,0.00358,0.00239,0.00496,0.00249,0.00233,19.0
136858,0.0,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,19.0
136859,0.0,0.02201,0.01469,0.21438,0.01385,0.01082,0.03089,0.00974,0.04003,0.02264,0.02958,0.03813,0.01797,0.01812,0.01390,0.01201,0.01167,0.02037,0.01111,0.01874,0.01148,0.01738,0.01379,0.01229,11.0


In [ ]:
df_test = df_num_topics.drop(columns=['top topic'])
df_test

,-1,1,10,11,12,13,14,15,16,17,18,19,2,20,21,22,23,3,4,5,6,7,8,9
0,0.0,0.00945,0.00632,0.00758,0.00655,0.00449,0.00836,0.00573,0.00673,0.01072,0.00707,0.00575,0.00705,0.00542,0.00840,0.00676,0.00476,0.01670,0.00687,0.00727,0.00842,0.00582,0.74201,0.00716
1,0.0,0.01887,0.01220,0.01462,0.01339,0.00875,0.01572,0.01166,0.01312,0.02039,0.01371,0.01111,0.01292,0.01021,0.01765,0.01388,0.00942,0.03098,0.01422,0.01408,0.01782,0.01150,0.32470,0.01466
2,0.0,0.01862,0.01219,0.01512,0.01311,0.00879,0.01669,0.01149,0.01330,0.02156,0.01378,0.01138,0.01373,0.01064,0.01721,0.01401,0.00941,0.03526,0.01418,0.01459,0.01718,0.01147,0.34985,0.01507
3,0.0,0.01214,0.00828,0.00954,0.00897,0.00605,0.01048,0.00808,0.00861,0.01334,0.00899,0.00743,0.00900,0.00706,0.01155,0.00950,0.00645,0.01945,0.00970,0.00955,0.01195,0.00769,0.78610,0.01009
4,0.0,0.01349,0.00905,0.01051,0.01005,0.00660,0.01131,0.00893,0.00953,0.01440,0.00995,0.00816,0.00955,0.00758,0.01313,0.01049,0.00711,0.02107,0.01091,0.01031,0.01364,0.00852,0.76463,0.01109
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136856,0.0,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
136857,0.0,0.00436,0.00369,0.00738,0.00325,0.00300,0.00416,0.00212,0.01276,0.00356,0.00776,0.59365,0.00333,0.00412,0.00283,0.00240,0.00312,0.00311,0.00232,0.00358,0.00239,0.00496,0.00249,0.00233
136858,0.0,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
136859,0.0,0.02201,0.01469,0.21438,0.01385,0.01082,0.03089,0.00974,0.04003,0.02264,0.02958,0.03813,0.01797,0.01812,0.01390,0.01201,0.01167,0.02037,0.01111,0.01874,0.01148,0.01738,0.01379,0.01229


In [ ]:
sum(df_test.sum(axis=1) == 0)

0

In [ ]:
# check data
df

,text,emoji_free_text,at_free_text,url_free_text,prepared_text,tokens
0,RT @Aiannucci: If you’re against Cancel Cultur...,RT @Aiannucci: If you’re against Cancel Cultur...,"RT If you’re against Cancel Culture, why are y...","RT If you’re against Cancel Culture, why are y...",you against Cancel Culture why are you banning...,"[you, against, Cancel, Culture, why, are, you,..."
7,@OliverDowden Cancel culture\n\nInvented by th...,@OliverDowden Cancel culture Invented by the l...,Cancel culture Invented by the likes of you Gr...,Cancel culture Invented by the likes of you Gr...,Cancel culture Invented the likes you Grow and...,"[Cancel, culture, Invented, the, likes, you, G..."
11,@Nichols9Aj @Kryn_Womble @BlameHush When the d...,@Nichols9Aj @Kryn_Womble @BlameHush When the d...,"When the dude is casually saying racial slurs,...","When the dude is casually saying racial slurs,...",When the dude casually saying racial slurs mak...,"[When, the, dude, casually, saying, racial, sl..."
12,RT @kiwcor3: @sociotalker Indonesia need cance...,RT @kiwcor3: @sociotalker Indonesia need cance...,RT Indonesia need cancel culture sometime,RT Indonesia need cancel culture sometime,Indonesia need cancel culture sometime,"[Indonesia, need, cancel, culture, sometime]"
18,Tune in and join the weekly Parenting Show on ...,Tune in and join the weekly Parenting Show on ...,Tune in and join the weekly Parenting Show on ...,Tune in and join the weekly Parenting Show on ...,Tune and join the weekly Parenting Show Thursd...,"[Tune, and, join, the, weekly, Parenting, Show..."
...,...,...,...,...,...,...
346721,@andybeega @SpaghettiKozak @soonergrunt @tmbhm...,@andybeega @SpaghettiKozak @soonergrunt @tmbhm...,"As I said earlier, gun to my head, I guess I'd...","As I said earlier, gun to my head, I guess I'd...",said earlier gun head guess rather Ukraine win...,"[said, earlier, gun, head, guess, rather, Ukra..."
346723,RT @SkyNews: Speaking at his annual news confe...,RT @SkyNews: Speaking at his annual news confe...,"RT Speaking at his annual news conference, Rus...","RT Speaking at his annual news conference, Rus...",Speaking his annual news conference Russian Pr...,"[Speaking, his, annual, news, conference, Russ..."
346724,@Morearighty It's easy. I'm a Russian immigran...,@Morearighty It's easy. I'm a Russian immigran...,It's easy. I'm a Russian immigrant who doesn't...,It's easy. I'm a Russian immigrant who doesn't...,easy Russian immigrant who doesn support Russi...,"[easy, Russian, immigrant, who, doesn, support..."
346725,RT @France24_en: @RobParsonsF24 The Wagner Gro...,RT @France24_en: @RobParsonsF24 The Wagner Gro...,RT The Wagner Group has caused controversy thr...,RT The Wagner Group has caused controversy thr...,The Wagner Group has caused controversy throug...,"[The, Wagner, Group, has, caused, controversy,..."


In [ ]:
df = df.reset_index(drop=True)
df

,text,emoji_free_text,at_free_text,url_free_text,prepared_text,tokens
0,RT @Aiannucci: If you’re against Cancel Cultur...,RT @Aiannucci: If you’re against Cancel Cultur...,"RT If you’re against Cancel Culture, why are y...","RT If you’re against Cancel Culture, why are y...",you against Cancel Culture why are you banning...,"[you, against, Cancel, Culture, why, are, you,..."
1,@OliverDowden Cancel culture\n\nInvented by th...,@OliverDowden Cancel culture Invented by the l...,Cancel culture Invented by the likes of you Gr...,Cancel culture Invented by the likes of you Gr...,Cancel culture Invented the likes you Grow and...,"[Cancel, culture, Invented, the, likes, you, G..."
2,@Nichols9Aj @Kryn_Womble @BlameHush When the d...,@Nichols9Aj @Kryn_Womble @BlameHush When the d...,"When the dude is casually saying racial slurs,...","When the dude is casually saying racial slurs,...",When the dude casually saying racial slurs mak...,"[When, the, dude, casually, saying, racial, sl..."
3,RT @kiwcor3: @sociotalker Indonesia need cance...,RT @kiwcor3: @sociotalker Indonesia need cance...,RT Indonesia need cancel culture sometime,RT Indonesia need cancel culture sometime,Indonesia need cancel culture sometime,"[Indonesia, need, cancel, culture, sometime]"
4,Tune in and join the weekly Parenting Show on ...,Tune in and join the weekly Parenting Show on ...,Tune in and join the weekly Parenting Show on ...,Tune in and join the weekly Parenting Show on ...,Tune and join the weekly Parenting Show Thursd...,"[Tune, and, join, the, weekly, Parenting, Show..."
...,...,...,...,...,...,...
136856,@andybeega @SpaghettiKozak @soonergrunt @tmbhm...,@andybeega @SpaghettiKozak @soonergrunt @tmbhm...,"As I said earlier, gun to my head, I guess I'd...","As I said earlier, gun to my head, I guess I'd...",said earlier gun head guess rather Ukraine win...,"[said, earlier, gun, head, guess, rather, Ukra..."
136857,RT @SkyNews: Speaking at his annual news confe...,RT @SkyNews: Speaking at his annual news confe...,"RT Speaking at his annual news conference, Rus...","RT Speaking at his annual news conference, Rus...",Speaking his annual news conference Russian Pr...,"[Speaking, his, annual, news, conference, Russ..."
136858,@Morearighty It's easy. I'm a Russian immigran...,@Morearighty It's easy. I'm a Russian immigran...,It's easy. I'm a Russian immigrant who doesn't...,It's easy. I'm a Russian immigrant who doesn't...,easy Russian immigrant who doesn support Russi...,"[easy, Russian, immigrant, who, doesn, support..."
136859,RT @France24_en: @RobParsonsF24 The Wagner Gro...,RT @France24_en: @RobParsonsF24 The Wagner Gro...,RT The Wagner Group has caused controversy thr...,RT The Wagner Group has caused controversy thr...,The Wagner Group has caused controversy throug...,"[The, Wagner, Group, has, caused, controversy,..."


In [ ]:
df_new = pd.concat([df, df_num_topics], axis=1)

In [ ]:
# check data
df_new

,text,emoji_free_text,at_free_text,url_free_text,prepared_text,tokens,-1,1,10,11,12,13,14,15,16,17,18,19,2,20,21,22,23,3,4,5,6,7,8,9,top topic
0,RT @Aiannucci: If you’re against Cancel Cultur...,RT @Aiannucci: If you’re against Cancel Cultur...,"RT If you’re against Cancel Culture, why are y...","RT If you’re against Cancel Culture, why are y...",you against Cancel Culture why are you banning...,"[you, against, Cancel, Culture, why, are, you,...",0.0,0.00945,0.00632,0.00758,0.00655,0.00449,0.00836,0.00573,0.00673,0.01072,0.00707,0.00575,0.00705,0.00542,0.00840,0.00676,0.00476,0.01670,0.00687,0.00727,0.00842,0.00582,0.74201,0.00716,8.0
1,@OliverDowden Cancel culture\n\nInvented by th...,@OliverDowden Cancel culture Invented by the l...,Cancel culture Invented by the likes of you Gr...,Cancel culture Invented by the likes of you Gr...,Cancel culture Invented the likes you Grow and...,"[Cancel, culture, Invented, the, likes, you, G...",0.0,0.01887,0.01220,0.01462,0.01339,0.00875,0.01572,0.01166,0.01312,0.02039,0.01371,0.01111,0.01292,0.01021,0.01765,0.01388,0.00942,0.03098,0.01422,0.01408,0.01782,0.01150,0.32470,0.01466,8.0
2,@Nichols9Aj @Kryn_Womble @BlameHush When the d...,@Nichols9Aj @Kryn_Womble @BlameHush When the d...,"When the dude is casually saying racial slurs,...","When the dude is casually saying racial slurs,...",When the dude casually saying racial slurs mak...,"[When, the, dude, casually, saying, racial, sl...",0.0,0.01862,0.01219,0.01512,0.01311,0.00879,0.01669,0.01149,0.01330,0.02156,0.01378,0.01138,0.01373,0.01064,0.01721,0.01401,0.00941,0.03526,0.01418,0.01459,0.01718,0.01147,0.34985,0.01507,8.0
3,RT @kiwcor3: @sociotalker Indonesia need cance...,RT @kiwcor3: @sociotalker Indonesia need cance...,RT Indonesia need cancel culture sometime,RT Indonesia need cancel culture sometime,Indonesia need cancel culture sometime,"[Indonesia, need, cancel, culture, sometime]",0.0,0.01214,0.00828,0.00954,0.00897,0.00605,0.01048,0.00808,0.00861,0.01334,0.00899,0.00743,0.00900,0.00706,0.01155,0.00950,0.00645,0.01945,0.00970,0.00955,0.01195,0.00769,0.78610,0.01009,8.0
4,Tune in and join the weekly Parenting Show on ...,Tune in and join the weekly Parenting Show on ...,Tune in and join the weekly Parenting Show on ...,Tune in and join the weekly Parenting Show on ...,Tune and join the weekly Parenting Show Thursd...,"[Tune, and, join, the, weekly, Parenting, Show...",0.0,0.01349,0.00905,0.01051,0.01005,0.00660,0.01131,0.00893,0.00953,0.01440,0.00995,0.00816,0.00955,0.00758,0.01313,0.01049,0.00711,0.02107,0.01091,0.01031,0.01364,0.00852,0.76463,0.01109,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136856,@andybeega @SpaghettiKozak @soonergrunt @tmbhm...,@andybeega @SpaghettiKozak @soonergrunt @tmbhm...,"As I said earlier, gun to my head, I guess I'd...","As I said earlier, gun to my head, I guess I'd...",said earlier gun head guess rather Ukraine win...,"[said, earlier, gun, head, guess, rather, Ukra...",0.0,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,19.0
136857,RT @SkyNews: Speaking at his annual news confe...,RT @SkyNews: Speaking at his annual news confe...,"RT Speaking at his annual news conference, Rus...","RT Speaking at his annual news conference, Rus...",Speaking his annual news conference Russian Pr...,"[Speaking, his, annual, news, conference, Russ...",0.0,0.00436,0.00369,0.00738,0.00325,0.00300,0.00416,0.00212,0.01276,0.00356,0.00776,0.59365,0.00333,0.00412,0.00283,0.00240,0.00312,0.00311,0.00232,0.00358,0.00239,0.00496,0.00249,0.00233,19.0
136858,@Morearighty It's easy. I'm a Russian immigran...,@Morearighty It's easy. I'm a Russian immigran...,It's easy. I'm a Russian immigrant who doesn't...,It's easy. I'm a Russian immigrant who doesn't...,easy Russian immigrant who doesn support Russi...,"[ea

In [ ]:
df_new = df_new.rename(columns={"index":"New_ID"})
df_new['New_ID'] = df_new.index + 1

In [ ]:
# check data
df_new

,text,emoji_free_text,at_free_text,url_free_text,prepared_text,tokens,-1,1,10,11,12,13,14,15,16,17,18,19,2,20,21,22,23,3,4,5,6,7,8,9,top topic,New_ID
0,RT @Aiannucci: If you’re against Cancel Cultur...,RT @Aiannucci: If you’re against Cancel Cultur...,"RT If you’re against Cancel Culture, why are y...","RT If you’re against Cancel Culture, why are y...",you against Cancel Culture why are you banning...,"[you, against, Cancel, Culture, why, are, you,...",0.0,0.00945,0.00632,0.00758,0.00655,0.00449,0.00836,0.00573,0.00673,0.01072,0.00707,0.00575,0.00705,0.00542,0.00840,0.00676,0.00476,0.01670,0.00687,0.00727,0.00842,0.00582,0.74201,0.00716,8.0,1
1,@OliverDowden Cancel culture\n\nInvented by th...,@OliverDowden Cancel culture Invented by the l...,Cancel culture Invented by the likes of you Gr...,Cancel culture Invented by the likes of you Gr...,Cancel culture Invented the likes you Grow and...,"[Cancel, culture, Invented, the, likes, you, G...",0.0,0.01887,0.01220,0.01462,0.01339,0.00875,0.01572,0.01166,0.01312,0.02039,0.01371,0.01111,0.01292,0.01021,0.01765,0.01388,0.00942,0.03098,0.01422,0.01408,0.01782,0.01150,0.32470,0.01466,8.0,2
2,@Nichols9Aj @Kryn_Womble @BlameHush When the d...,@Nichols9Aj @Kryn_Womble @BlameHush When the d...,"When the dude is casually saying racial slurs,...","When the dude is casually saying racial slurs,...",When the dude casually saying racial slurs mak...,"[When, the, dude, casually, saying, racial, sl...",0.0,0.01862,0.01219,0.01512,0.01311,0.00879,0.01669,0.01149,0.01330,0.02156,0.01378,0.01138,0.01373,0.01064,0.01721,0.01401,0.00941,0.03526,0.01418,0.01459,0.01718,0.01147,0.34985,0.01507,8.0,3
3,RT @kiwcor3: @sociotalker Indonesia need cance...,RT @kiwcor3: @sociotalker Indonesia need cance...,RT Indonesia need cancel culture sometime,RT Indonesia need cancel culture sometime,Indonesia need cancel culture sometime,"[Indonesia, need, cancel, culture, sometime]",0.0,0.01214,0.00828,0.00954,0.00897,0.00605,0.01048,0.00808,0.00861,0.01334,0.00899,0.00743,0.00900,0.00706,0.01155,0.00950,0.00645,0.01945,0.00970,0.00955,0.01195,0.00769,0.78610,0.01009,8.0,4
4,Tune in and join the weekly Parenting Show on ...,Tune in and join the weekly Parenting Show on ...,Tune in and join the weekly Parenting Show on ...,Tune in and join the weekly Parenting Show on ...,Tune and join the weekly Parenting Show Thursd...,"[Tune, and, join, the, weekly, Parenting, Show...",0.0,0.01349,0.00905,0.01051,0.01005,0.00660,0.01131,0.00893,0.00953,0.01440,0.00995,0.00816,0.00955,0.00758,0.01313,0.01049,0.00711,0.02107,0.01091,0.01031,0.01364,0.00852,0.76463,0.01109,8.0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136856,@andybeega @SpaghettiKozak @soonergrunt @tmbhm...,@andybeega @SpaghettiKozak @soonergrunt @tmbhm...,"As I said earlier, gun to my head, I guess I'd...","As I said earlier, gun to my head, I guess I'd...",said earlier gun head guess rather Ukraine win...,"[said, earlier, gun, head, guess, rather, Ukra...",0.0,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,19.0,136857
136857,RT @SkyNews: Speaking at his annual news confe...,RT @SkyNews: Speaking at his annual news confe...,"RT Speaking at his annual news conference, Rus...","RT Speaking at his annual news conference, Rus...",Speaking his annual news conference Russian Pr...,"[Speaking, his, annual, news, conference, Russ...",0.0,0.00436,0.00369,0.00738,0.00325,0.00300,0.00416,0.00212,0.01276,0.00356,0.00776,0.59365,0.00333,0.00412,0.00283,0.00240,0.00312,0.00311,0.00232,0.00358,0.00239,0.00496,0.00249,0.00233,19.0,136858
136858,@Morearighty It's easy. I'm a Russian immigran...,@Morearighty It's easy. I'm a Russian immigran...,It's easy. I'm a Russian immigrant who doesn't...,It's easy. I'm a Russian immigrant who doesn't...,easy Russian immigr

### Perform some Validation if the merge was successful

In [ ]:
pd.set_option('display.max_colwidth', None)

In [ ]:
# Test 1

In [ ]:
df_new[55:56]

,text,emoji_free_text,at_free_text,url_free_text,prepared_text,tokens,-1,1,10,11,12,13,14,15,16,17,18,19,2,20,21,22,23,3,4,5,6,7,8,9,top topic,New_ID
55,@trussliz No portrait of the queen??? Leftist cancel culture Commie.,@trussliz No portrait of the queen??? Leftist cancel culture Commie.,No portrait of the queen??? Leftist cancel culture Commie.,No portrait of the queen??? Leftist cancel culture Commie.,portrait the queen Leftist cancel culture Commie,"[portrait, the, queen, Leftist, cancel, culture, Commie]",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,8.0,56


In [ ]:
df_new[55:56]['text']

55    @trussliz No portrait of the queen??? Leftist cancel culture Commie.
Name: text, dtype: object

In [ ]:
topics[55] + 1 #  plus one as I added always 1 as a topic key

7

In [ ]:
probs[55] # order in dataframe is different due to column order

array([1.76561369e-308, 1.29688246e-308, 3.14131794e-308, 1.27098290e-308,
       1.35463956e-308, 1.56321673e-308, 1.07370597e-308, 1.00000000e+000,
       1.34094424e-308, 1.16516718e-308, 1.40222874e-308, 1.21572486e-308,
       8.27982333e-309, 1.55024082e-308, 1.06620017e-308, 1.24160409e-308,
       2.00913081e-308, 1.30186698e-308, 1.05778705e-308, 9.99376921e-309,
       1.56542954e-308, 1.26460371e-308, 8.77294783e-309])

In [ ]:
# Test 2

In [ ]:
df_new[11656:11657]

,text,emoji_free_text,at_free_text,url_free_text,prepared_text,tokens,-1,1,10,11,12,13,14,15,16,17,18,19,2,20,21,22,23,3,4,5,6,7,8,9,top topic,New_ID
11656,RT @PSYOP_warrior: @mail_american @KAG2020DJT2020 Climate change has been occurring for 4.5 BILLION years.,RT @PSYOP_warrior: @mail_american @KAG2020DJT2020 Climate change has been occurring for 4.5 BILLION years.,RT Climate change has been occurring for 4.5 BILLION years.,RT Climate change has been occurring for 4.5 BILLION years.,Climate change has been occurring for BILLION years,"[Climate, change, has, been, occurring, for, BILLION, years]",0.0,0.00493,0.93027,0.00304,0.00337,0.00503,0.00278,0.0023,0.00412,0.0031,0.00528,0.00315,0.00289,0.00316,0.00253,0.00206,0.00324,0.00239,0.00185,0.00335,0.00227,0.00459,0.00241,0.00187,10.0,11657


In [ ]:
df_new[11656:11657]['text']

11656    RT @PSYOP_warrior: @mail_american @KAG2020DJT2020 Climate change has been occurring for 4.5 BILLION years.
Name: text, dtype: object

In [ ]:
topics[11656] + 1 #  plus one as I added always 1 as a topic key

10

In [ ]:
probs[11656] # order in dataframe is different due to column order

array([0.00493384, 0.00288808, 0.00239282, 0.0018524 , 0.00334527,
       0.00226825, 0.00458721, 0.00241497, 0.00187346, 0.93027295,
       0.00303983, 0.00337146, 0.00503289, 0.00278466, 0.00229559,
       0.00411576, 0.00309875, 0.00528169, 0.00315269, 0.00316442,
       0.00253361, 0.00205704, 0.00324237])

In [ ]:
# Test 3

In [ ]:
df_new[29856:29857]

,text,emoji_free_text,at_free_text,url_free_text,prepared_text,tokens,-1,1,10,11,12,13,14,15,16,17,18,19,2,20,21,22,23,3,4,5,6,7,8,9,top topic,New_ID
29856,"If Elon states 20 million Teslas in 2030 and people predict EV share at cough, cough 24% then Tesla will have 83% of the EV market 😀 ⁦@CNBC⁩ . Tesla delivered 936,172 electric vehicles in 2021, with the fourth-quarter setting a new record @CNBC https://t.co/NYUW8GOFOi","If Elon states 20 million Teslas in 2030 and people predict EV share at cough, cough 24% then Tesla will have 83% of the EV market ⁦@CNBC⁩ . Tesla delivered 936,172 electric vehicles in 2021, with the fourth-quarter setting a new record @CNBC https://t.co/NYUW8GOFOi","If Elon states 20 million Teslas in 2030 and people predict EV share at cough, cough 24% then Tesla will have 83% of the EV market . Tesla delivered 936,172 electric vehicles in 2021, with the fourth-quarter setting a new record https://t.co/NYUW8GOFOi","If Elon states 20 million Teslas in 2030 and people predict EV share at cough, cough 24% then Tesla will have 83% of the EV market . Tesla delivered 936,172 electric vehicles in 2021, with the fourth-quarter setting a new record",Elon states million Teslas and people predict share cough cough then Tesla will have the market Tesla delivered electric vehicles with the fourth quarter setting new record,"[Elon, states, million, Teslas, and, people, predict, share, cough, cough, then, Tesla, will, have, the, market, Tesla, delivered, electric, vehicles, with, the, fourth, quarter, setting, new, record]",0.0,0.01748,0.02697,0.01272,0.0195,0.14927,0.01086,0.01239,0.01835,0.01155,0.01969,0.01539,0.01048,0.01229,0.01263,0.0104,0.03212,0.0096,0.0098,0.01383,0.01136,0.03275,0.00961,0.0093,13.0,29857


In [ ]:
print(df_new[29856:29857]['text'])

29856    If Elon states 20 million Teslas in 2030 and people predict EV share at cough, cough 24% then Tesla will have 83% of the EV market 😀 ⁦@CNBC⁩ .  Tesla delivered 936,172 electric vehicles in 2021, with the fourth-quarter setting a new record @CNBC https://t.co/NYUW8GOFOi
Name: text, dtype: object


In [ ]:
topics[29856] + 1 #  plus one as I added always 1 as a topic key

13

In [ ]:
probs[29856] # order in dataframe is different due to column order

array([0.0174762 , 0.01048457, 0.00959609, 0.0098016 , 0.01383012,
       0.01135641, 0.03274771, 0.00960747, 0.00929739, 0.02696617,
       0.01271803, 0.01949552, 0.14927056, 0.01086   , 0.0123908 ,
       0.01835218, 0.01154973, 0.01968601, 0.01539104, 0.01228818,
       0.01262689, 0.01040275, 0.03211767])

In [ ]:
pd.set_option('max_colwidth', 60)

In [ ]:
# # not always needed
# df_new = df_new.drop(columns=['topic_theme', 'author_id', 'created_at', 'geo', 'tweet_id', 'lang', 'like_count', 'quote_count', 'reply_count', 'retweet_count', 'source', 'retweeted'])

In [ ]:
# check data
df_new

,text,emoji_free_text,at_free_text,url_free_text,prepared_text,tokens,-1,1,10,11,12,13,14,15,16,17,18,19,2,20,21,22,23,3,4,5,6,7,8,9,top topic,New_ID
0,"RT @Aiannucci: If you’re against Cancel Culture, why are...","RT @Aiannucci: If you’re against Cancel Culture, why are...","RT If you’re against Cancel Culture, why are you banning...","RT If you’re against Cancel Culture, why are you banning...",you against Cancel Culture why are you banning demonstra...,"[you, against, Cancel, Culture, why, are, you, banning, ...",0.0,0.00945,0.00632,0.00758,0.00655,0.00449,0.00836,0.00573,0.00673,0.01072,0.00707,0.00575,0.00705,0.00542,0.00840,0.00676,0.00476,0.01670,0.00687,0.00727,0.00842,0.00582,0.74201,0.00716,8.0,1
1,@OliverDowden Cancel culture\n\nInvented by the likes of...,@OliverDowden Cancel culture Invented by the likes of yo...,Cancel culture Invented by the likes of you Grow up and ...,Cancel culture Invented by the likes of you Grow up and ...,Cancel culture Invented the likes you Grow and stop fcki...,"[Cancel, culture, Invented, the, likes, you, Grow, and, ...",0.0,0.01887,0.01220,0.01462,0.01339,0.00875,0.01572,0.01166,0.01312,0.02039,0.01371,0.01111,0.01292,0.01021,0.01765,0.01388,0.00942,0.03098,0.01422,0.01408,0.01782,0.01150,0.32470,0.01466,8.0,2
2,@Nichols9Aj @Kryn_Womble @BlameHush When the dude is cas...,@Nichols9Aj @Kryn_Womble @BlameHush When the dude is cas...,"When the dude is casually saying racial slurs, making tr...","When the dude is casually saying racial slurs, making tr...",When the dude casually saying racial slurs making transp...,"[When, the, dude, casually, saying, racial, slurs, makin...",0.0,0.01862,0.01219,0.01512,0.01311,0.00879,0.01669,0.01149,0.01330,0.02156,0.01378,0.01138,0.01373,0.01064,0.01721,0.01401,0.00941,0.03526,0.01418,0.01459,0.01718,0.01147,0.34985,0.01507,8.0,3
3,RT @kiwcor3: @sociotalker Indonesia need cancel culture ...,RT @kiwcor3: @sociotalker Indonesia need cancel culture ...,RT Indonesia need cancel culture sometime,RT Indonesia need cancel culture sometime,Indonesia need cancel culture sometime,"[Indonesia, need, cancel, culture, sometime]",0.0,0.01214,0.00828,0.00954,0.00897,0.00605,0.01048,0.00808,0.00861,0.01334,0.00899,0.00743,0.00900,0.00706,0.01155,0.00950,0.00645,0.01945,0.00970,0.00955,0.01195,0.00769,0.78610,0.01009,8.0,4
4,Tune in and join the weekly Parenting Show on Thursdays ...,Tune in and join the weekly Parenting Show on Thursdays ...,Tune in and join the weekly Parenting Show on Thursdays ...,Tune in and join the weekly Parenting Show on Thursdays ...,Tune and join the weekly Parenting Show Thursdays LagosT...,"[Tune, and, join, the, weekly, Parenting, Show, Thursday...",0.0,0.01349,0.00905,0.01051,0.01005,0.00660,0.01131,0.00893,0.00953,0.01440,0.00995,0.00816,0.00955,0.00758,0.01313,0.01049,0.00711,0.02107,0.01091,0.01031,0.01364,0.00852,0.76463,0.01109,8.0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136856,@andybeega @SpaghettiKozak @soonergrunt @tmbhmltn @Casti...,@andybeega @SpaghettiKozak @soonergrunt @tmbhmltn @Casti...,"As I said earlier, gun to my head, I guess I'd rather Uk...","As I said earlier, gun to my head, I guess I'd rather Uk...",said earlier gun head guess rather Ukraine win the confl...,"[said, earlier, gun, head, guess, rather, Ukraine, win, ...",0.0,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,19.0,136857
136857,"RT @SkyNews: Speaking at his annual news conference, Rus...","RT @SkyNews: Speaking at his annual news conference, Rus...","RT Speaking at his annual news conference, Russian Presi...","RT Speaking at his annual news conference, Russian Presi...",Speaking his annual news conference Russian President Vl...,"[Speaking, his, annual, news, conference, Russian, Presi...",0.0,0.00436,0.00369,0.00738,0.00325,0.

In [ ]:
# Validation looks fine - code seem to work
# store list to loca disc
df_new.to_csv('data_complete_all_types_predictions.csv')